In [1]:
import os
import logging
import numpy as np
import xarray as xr
from dask.diagnostics import ProgressBar
from dask.distributed import Client

# Set global variables
PROJECT = r"D:\World Bank\Paper - Child mortality and Climate Shocks"
OUTPUTS = rf"{PROJECT}\Outputs"
DATA = rf"{PROJECT}\Data"
DATA_IN = rf"{DATA}\Data_in"
DATA_PROC = rf"{DATA}\Data_proc"
DATA_OUT = rf"{DATA}\Data_out"
ERA5_DATA = r"D:\Datasets\ERA5 Reanalysis\monthly-single-levels"

In [ ]:
client = Client()
client

In [ ]:
climate_shocks_v9 = xr.open_dataset(
        r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_proc\Climate_shocks_v9.nc",
        chunks={"lat": 60, "lon": 60, "time": 5}
    ).drop_vars("month")
lat_tgt = climate_shocks_v9.lat
lon_tgt = climate_shocks_v9.lon

# Interpolate and store the datasets
tasks = []
for var in ["hd35", "hd40", "fd", "id"]:
    ds_var = xr.open_dataset(
        rf"{DATA_IN}\Climate Data\timeseries-{var}-monthly-mean_era_monthly_era5-0.5x0.5-timeseries_mean_1950-2020.nc",
        chunks={"lat": 60, "lon": 60, "time": 5})\
        .sel(bnds=0, time=slice("1990-01-01", "2020-12-01"))\
        .drop_vars("bnds")\
        [f"timeseries-{var}-monthly-mean"].rename(var)
    ds_var = (ds_var / np.timedelta64(1, 'D')).astype(int)
    # ds_var = ds_var.transpose("lat", "lon", "time")
    
    ds_var = ds_var.interp(
        lat=lat_tgt,
        lon=lon_tgt,
        method="linear",
    )

    # Save
    outname = rf"{DATA_PROC}\{var}_interpolated.nc"
    ds_var.to_netcdf(outname, encoding={var: {"zlib": True, "complevel": 6}}, compute=True)
    print(f"Saved {var} to {outname}")


In [ ]:
climate_shocks_v9 = xr.open_dataset(
        r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_proc\Climate_shocks_v9.nc",
        chunks={"lat": 200, "lon": 360, "time": None}
    ).drop_vars("month")

# Load the datasets and combine them
dss = [climate_shocks_v9]
for var in ["hd35", "hd40", "fd", "id"]:
    ds_var = xr.open_dataset(
        rf"{DATA_PROC}\{var}_interpolated.nc",
        chunks={"lat": 200, "lon": 360, "time": None}
    )
    dss += [ds_var]

ds = xr.combine_by_coords(dss)

# Export
out = rf"{DATA_PROC}/Climate_shocks_v9d.nc"
encoding = {
    var: {"zlib": True, "complevel": 6} for var in ds.data_vars
}
ds.to_netcdf(
    out,
    encoding=encoding,
    
)


In [ ]:
climate_shocks_v9